# WordCount sample application

In this notebook, you will download a text file and then use the Spark cluster run by Azure Databricks to count each distinct word in the file.

This sample application is meant to provide you with template code to access the Azure Data Lake (blob) storage and Cosmos DB. The blob storage is accessed to read the text file and to write onto it the word count. Cosmos DB is also used to store the word count.

## Attach notebook to you cluster

Before executing any cells in the notebook, you need to attach it to your cluster. In the notebook's toolbar, select the drop down arrow next to Detached, and then select your cluster under Attach to.

![Detach is expanded in the notebook toolbar, and the cluster is highlighted under Attach to.](https://github.com/alagala/labs/raw/master/azure/databricks/media/databricks-attach-notebook.png "Attach notebook")

## Download the sample text file on the cluster local store

In [4]:
%sh
wget -P /tmp https://raw.githubusercontent.com/alagala/labs/master/azure/databricks/notebooks/data/sample.txt

## Connect to the Azure Data Lake storage

The snippet of code below connects to the Azure Data Lake (blob) storage and uploads the sample text file. Then, it lists all files available on the blob storage.

For more information, refer to the [Azure Databricks documentation](https://docs.databricks.com/spark/latest/data-sources/azure/azure-datalake-gen2.html).

> **NOTE**: For this hands-on lab, the endpoint and master key have already been added to the Azure Key Vault service, so you will retrieve the values from there using `dbutils.secrets.get()`.

In [6]:
# Variable declarations. These will be accessible by any calling notebook.
keyVaultScope = "key-vault-secrets"
adlsAccountName = dbutils.secrets.get(keyVaultScope, "ADLS-Gen2-Account-Name")
adlsAccountKey = dbutils.secrets.get(keyVaultScope, "ADLS-Gen2-Account-Key")

abfsUri = "abfss://wordcount@" + adlsAccountName + ".dfs.core.windows.net/"

# Since the hierarchical namespace is enabled for the storage account, we must initialize a filesystem before we can access it.
spark.conf.set("fs.azure.account.key." + adlsAccountName + ".dfs.core.windows.net", adlsAccountKey)
spark.conf.set("fs.azure.createRemoteFileSystemDuringInitialization", "true")
dbutils.fs.ls(abfsUri)
spark.conf.set("fs.azure.createRemoteFileSystemDuringInitialization", "false")

dbutils.fs.cp("file:///tmp/sample.txt", abfsUri)

dbutils.fs.ls(abfsUri)

## Run the word count program

In [8]:
from pyspark.sql.functions import regexp_replace
from pyspark.sql.functions import lower
from pyspark.sql.functions import trim
from pyspark.sql.functions import split
from pyspark.sql.functions import explode
from pyspark.sql.functions import desc

df = spark.read.text(abfsUri + "sample.txt")

wordcount = df.select(
  # Remove punctuation and explode each line of text into a row per word.
  explode(split(trim(lower(regexp_replace(df.value, "[,.!?:;]", ""))), " ")).alias("word")
).groupBy("word").count().orderBy(desc("count"))

wordcount.show(10)

## Connect to Cosmos DB

Using the Azure Cosmos DB Spark Connector, you will now use Cosmos DB as an input source to retrieve a sample of transaction data.

In order to write to Cosmos DB, you need to first create a configuration object that contains the configuration information. If you are curious, read the [configuration reference](https://github.com/Azure/azure-cosmosdb-spark/wiki/Configuration-references) for details on all of the options. 

The core items you need to provide are:

  - **Endpoint**: Your Cosmos DB url (i.e. https://youraccount.documents.azure.com:443/).
  - **Masterkey**: The primary or secondary key string for you Cosmos DB account.
  - **Database**: The name of the database.
  - **Collection**: The name of the collection that you wish to query.

> **NOTE**: For this hands-on lab, the endpoint and master key have already been added to the Azure Key Vault service, so you will retrieve the values from there using `dbutils.secrets.get()`. Remember to replace `<DATABASE_ID>` with the actual name of the database that you used when creating it.

In [10]:
# https://github.com/Azure/azure-cosmosdb-spark/wiki/Configuration-references
# https://docs.microsoft.com/en-us/azure/cosmos-db/spark-connector

# Write configuration
writeConfig = {
    "Endpoint" : dbutils.secrets.get(keyVaultScope, "Cosmos-DB-URI"),
    "Masterkey" : dbutils.secrets.get(keyVaultScope, "Cosmos-DB-Key"),
    "Database" : "<DATABASE_ID>", # TODO: replace <DATABASE_ID> with the actual name you used when creating the database in Cosmos DB
    "Collection" : "words",
    "Upsert" : "true"
}

# Write to Cosmos DB from the flights DataFrame
wordcount.write.format("com.microsoft.azure.cosmosdb.spark").options(**writeConfig).save()

## Persist the transaction data to an Azure Databricks Delta table

[Databricks Delta](https://docs.databricks.com/delta/delta-intro.html) delivers a powerful transactional storage layer by harnessing the power of Apache Spark and Databricks DBFS. The core abstraction of Delta is an optimized Spark table that:

  - Stores data as Parquet files in DBFS.
  - Maintains a transaction log that efficiently tracks changes to the table.

You read and write data stored in the delta format using the same familiar Apache Spark SQL batch and streaming APIs that you use to work with Hive tables and DBFS directories. With the addition of the transaction log and other enhancements, Delta offers significant benefits:

  - **ACID transactions**
    - Multiple writers can simultaneously modify a dataset and see consistent views. For qualifications, see Multi-cluster writes.
    - Writers can modify a dataset without interfering with jobs reading the dataset.
  - **Fast read access**
    - Automatic file management organizes data into large files that can be read efficiently.
    - Statistics enable speeding up reads by 10-100x and and data skipping avoids reading irrelevant information.
    
To create your transactions Delta table, you will first write the word count DataFrame to a folder in Azure Data Lake (blob) storage in Databricks Delta format.

Let's break down the command in the cell below before running it.

  - `mode("overwrite")`: This tells the write operation to overwrite any existing Delta table stored in the specified location.
  - `format("delta")`: To save the data in Delta format, you specify "delta" with the `format()` option of the `write` command.
  - `partitionBy()`: When creating a new Delta table, you can **optionally** specify partition columns. Partitioning is used to speed up queries or DML that have predicates involving the partition columns.
  - `save()`: The `save` command accepts a location, which is where the underlying files for the Delta table will be stored. In our case, this is a location in the Azure Data Lake storage, which you will provide using the abfs URI.

In [12]:
wordcount.write.mode("overwrite").format("delta").save(abfsUri + "delta")

Now that you have saved the clean transaction data into a Azure Data Lake storage blob location in Delta format, you can create a Databricks global table which is backed by the Delta location you created above. Notice that the `LOCATION` specified in the `CREATE TABLE` query is the same as what you used to write the word count data in Delta format above. Doing this allows the table in the Hive metastore to automatically inherit the schema, partitioning, and table properties of the existing data.

**IMPORTANT**: You will need to add the name of your Azure Data Lake storage account into the location value below, before running the cell.

In [14]:
%sql
CREATE TABLE wordcount
USING DELTA
-- TODO: Replace <STORAGE_ACCOUNT_NAME> with the actual Azure Data Lake storage account name (eg. datainsightspoc27450)
LOCATION 'abfss://wordcount@<STORAGE_ACCOUNT_NAME>.dfs.core.windows.net/delta'

You can use the `DESCRIBE DETAIL` SQL command to view information about schema, partitioning, table size, and so on.

In [16]:
%sql
DESCRIBE DETAIL wordcount

Finally, you can use Spark SQL to query records in the Hive table.

In [18]:
%sql
SELECT * FROM wordcount
ORDER BY count DESC
LIMIT 10